# Бейзлайн модель для обнаружения паттернов КВД и КПД

In [ ]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import os


In [ ]:
# Конфигурация
DATA_PATH = "../data/raw/"
WINDOW_SIZE = 60   # точек в окне
STEP = 10         # шаг окна
MIN_DURATION = 4  # часов для recovery



In [ ]:


# ---------------------------------------------------
# 1. Загрузка данных и разметки
# ---------------------------------------------------
def load_data(file_id, is_test=False):
    """Загрузка временного ряда по ID файла"""
    # Определяем путь в зависимости от типа данных
    folder = "test" if is_test else "Task 21"
    path = os.path.join(DATA_PATH, folder, file_id)
    
    if not os.path.exists(path):
        print(f"Файл {file_id} не найден в папке {folder}.")
        return None
    
    df = pd.read_csv(path, sep='\t', header=None, names=['time', 'pressure'])
    return df.sort_values('time').reset_index(drop=True)


In [ ]:

# ---------------------------------------------------
# 2. Предобработка данных
# ---------------------------------------------------
def preprocess_series(df):
    """Сглаживание и нормализация"""
    df['pressure'] = df['pressure'].rolling(5, center=True, min_periods=1).mean()
    df['pressure'] = (df['pressure'] - df['pressure'].min()) / \
                    (df['pressure'].max() - df['pressure'].min())
    return df

In [ ]:

# ---------------------------------------------------
# 3. Генерация признаков для окон
# ---------------------------------------------------
from scipy.fftpack import fft
from scipy.integrate import simpson  # Используем simpson вместо simps

def get_window_features(window):
    """Извлечение признаков из окна"""
    time = window['time'].values
    pressure = window['pressure'].values
    
    # Базовые признаки
    features = {
        'slope': np.polyfit(time, pressure, 1)[0],
        'mean': np.mean(pressure),
        'std': np.std(pressure),
        'max_diff': np.max(pressure) - np.min(pressure),
        'duration': time[-1] - time[0],
        'starts_with_max': int(pressure[0] == np.max(pressure))  # Новый признак
    }
    
    # Частотные признаки (преобразование Фурье)
    fft_values = np.abs(fft(pressure))
    features['fft_peak'] = np.max(fft_values)
    features['fft_mean'] = np.mean(fft_values)
    
    # Производные
    first_derivative = np.gradient(pressure, time)
    second_derivative = np.gradient(first_derivative, time)
    features['first_derivative_max'] = np.max(first_derivative)
    features['second_derivative_max'] = np.max(second_derivative)
    
    # Интеграл (площадь под кривой)
    features['integral'] = simpson(pressure, time)
    
    # Автокорреляция (лаги 1, 2, 3)
    for lag in [1, 2, 3]:
        features[f'autocorr_lag_{lag}'] = np.corrcoef(pressure[:-lag], pressure[lag:])[0, 1]
    
    return pd.Series(features)

In [ ]:

# ---------------------------------------------------
# 4. Подготовка датасета для обучения
# ---------------------------------------------------
def create_dataset(file_ids, gt):
    """Создание размеченного датасета с проверкой существования файлов"""
    features = []
    targets = []
    
    # Проверка наличия файлов в Task 21
    valid_files = []
    for file_id in file_ids:
        path = os.path.join(DATA_PATH, "Task 21", file_id)
        if not os.path.exists(path):
            print(f"Файл {file_id} из разметки отсутствует в Task 21 и будет пропущен.")
        else:
            valid_files.append(file_id)
    
    # Обработка только валидных файлов
    for file_id in tqdm(valid_files):
        df = load_data(file_id, is_test=False)
        if df is None:
            continue
        
        df = preprocess_series(df)
        
        # Получение разметки для файла
        file_gt = gt[gt['file'] == file_id].iloc[0]
        recovery = eval(file_gt['recovery'])
        drop = eval(file_gt['drop'])
        
        # Скользящее окно
        for i in range(0, len(df)-WINDOW_SIZE, STEP):
            window = df.iloc[i:i+WINDOW_SIZE]
            window_features = get_window_features(window)
            
            # Проверка попадания в разметку
            target = 0
            start = window['time'].iloc[0]
            end = window['time'].iloc[-1]
            
            for interval in recovery:
                if interval[0] <= start and end <= interval[1]:
                    target = 1  # recovery
            for interval in drop:
                if interval[0] <= start and end <= interval[1]:
                    target = 2  # drop
                    
            features.append(window_features)
            targets.append(target)
    
    return pd.DataFrame(features), np.array(targets)

In [ ]:
# Создание датасета
gt = pd.read_csv(os.path.join(DATA_PATH, "ground_truth.csv"))
train_files = gt['file']
X, y = create_dataset(train_files, gt)

In [ ]:
# ---------------------------------------------------
# 5. Обучение модели с кросс-валидацией
# ---------------------------------------------------
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

# Настройки модели с лучшими параметрами
model_params = {
    'iterations': 1500,
    'learning_rate': 0.1,
    'depth': 6,
    'loss_function': 'MultiClass',
    'class_weights': [1, 20, 20],  # Увеличенные веса для recovery и drop
    'task_type': 'CPU',  # Для GPU: 'GPU'
    'verbose': 200
}

# Кросс-валидация
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
results = []

for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(**model_params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
    
    # Прогноз и метрики
    y_pred = model.predict(X_val)
    results.append(classification_report(y_val, y_pred, target_names=['background', 'recovery', 'drop']))



In [ ]:
# Вывод метрик
for i, report in enumerate(results):
    print(f"Fold {i+1}:\n{report}\n{'-'*50}")

In [ ]:
# Обучение финальной модели на всех данных
model = CatBoostClassifier(**model_params)
model.fit(X, y)

# Сохранение финальной модели
#model.save_model('model+.cbm')

In [ ]:

# ---------------------------------------------------
# 6. Предсказание и постобработка
# ---------------------------------------------------
def predict_intervals(model, file_id, is_test=False):
    """Предсказание интервалов для одного файла"""
    df = load_data(file_id, is_test=is_test)
    if df is None:
        return {'recovery': [], 'drop': []}
    
    df = preprocess_series(df)
    
    # Прогноз для всего ряда
    window_preds = []
    for i in range(0, len(df)-WINDOW_SIZE, STEP):
        window = df.iloc[i:i+WINDOW_SIZE]
        features = get_window_features(window)
        pred = model.predict(pd.DataFrame([features]))[0]
        proba = model.predict_proba(pd.DataFrame([features]))[0][pred]
        window_preds.append( (window['time'].iloc[0], window['time'].iloc[-1], pred, proba) )
    
    # Фильтрация по вероятности
    filtered = [ (start, end, label) for start, end, label, proba in window_preds if proba > 0.7 ]
    
    # Объединение интервалов
    merged = []
    current = None
    for start, end, label in filtered:
        if label == 0:
            continue
        if current and current[2] == label and current[1] >= start - 2:  # Зазор 2 часа
            current = (current[0], end, label)
        else:
            if current:
                merged.append(current)
            current = (start, end, label)
    if current:
        merged.append(current)
    
    
    # Оставляем только 1 лучший drop
    drop_intervals = [i for i in merged if i[2] == 2]
    if drop_intervals:
        drop_intervals = [max(drop_intervals, key=lambda x: x[1] - x[0])]  # Только 1 самый длинный

    # Перезапись отфильтрованных drop в merged
    merged = [i for i in merged if i[2] != 2] + drop_intervals
    
    # Оставляем только 1 самый длинный recovery
    recovery_intervals = [i for i in merged if i[2] == 1]
    if recovery_intervals:
        recovery_intervals = [max(recovery_intervals, key=lambda x: x[1] - x[0])]  # Только 1 самый длинный

    # Перезапись отфильтрованных recovery в merged
    merged = [i for i in merged if i[2] != 1] + recovery_intervals
    

    
    # Фильтрация по длительности и физическим ограничениям
    result = {'recovery': [], 'drop': []}
    for start, end, label in merged:
        duration = end - start
        if label == 1 and duration >= 6:  # Минимум 6 часов для recovery
            result['recovery'].append([start, end])
        elif label == 2 and duration >= 10:  # Минимум 10 часов для drop
            # Проверка, что drop начинается с максимума давления
            window = df[(df['time'] >= start) & (df['time'] <= end)]
            if window['pressure'].iloc[0] == window['pressure'].max():
                result['drop'].append([start, end])
    
    return result



In [ ]:

# ---------------------------------------------------
# 7. Визуализация результатов
# ---------------------------------------------------
def plot_comparison(file_id, pred_intervals, gt_entry):
    """
    Визуализация предсказаний и истинных интервалов.
    
    :param file_id: ID файла (строка).
    :param pred_intervals: Предсказанные интервалы (словарь с ключами 'recovery' и 'drop').
    :param gt_entry: Строка из ground_truth.csv с истинными интервалами.
    """
    # Загрузка данных
    df = load_data(file_id)
    if df is None:
        print(f"Файл {file_id} не найден.")
        return
    
    # Создание двух графиков
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    
    # График 1: Предсказанные интервалы
    ax1.plot(df['time'], df['pressure'], label='Давление', color='blue')
    
    # Предсказанные recovery
    for interval in pred_intervals['recovery']:
        start, end = interval
        ax1.axvspan(start, end, alpha=0.2, color='green')
    
    # Предсказанные drop
    for interval in pred_intervals['drop']:
        start, end = interval
        ax1.axvspan(start, end, alpha=0.2, color='red')
    
    ax1.set_title(f"Предсказанные интервалы для {file_id}")
    ax1.set_ylabel("Нормализованное давление")
    ax1.legend(loc='upper right')
    ax1.grid(True)
    
    # График 2: Истинные интервалы
    ax2.plot(df['time'], df['pressure'], label='Давление', color='blue')
    
    # Истинные recovery
    for interval in eval(gt_entry['recovery']):
        ax2.axvspan(interval[0], interval[1], alpha=0.2, color='green', label='True Recovery')
    
    # Истинные drop
    for interval in eval(gt_entry['drop']):
        ax2.axvspan(interval[0], interval[1], alpha=0.2, color='red', label='True Drop')
    
    ax2.set_title(f"Истинные интервалы для {file_id}")
    ax2.set_xlabel("Время (часы)")
    ax2.set_ylabel("Нормализованное давление")
    ax2.legend(loc='upper right')
    ax2.grid(True)
    
    # Удаление дубликатов в легенде
    handles, labels = ax2.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(), loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Пример использования
test_file = gt['file'].iloc[23]

predicted = predict_intervals(model, test_file)
gt_entry = gt[gt['file'] == test_file].iloc[0]

plot_comparison(test_file, predicted, gt_entry)

print(predicted)

In [ ]:

# ---------------------------------------------------
# 8. Генерация сабмита
# ---------------------------------------------------
def generate_submission(model, test_files, output_path):
    """Генерация файла для отправки на платформу"""
    submission = []
    
    for file_path in tqdm(test_files):
        # Извлекаем только имя файла без пути
        file_name = os.path.basename(file_path)
        
        # Предсказание с флагом is_test=True
        preds = predict_intervals(model, file_name, is_test=True)
        
        submission.append({
            'file': file_name,
            'recovery': str(preds['recovery']).replace(' ', ''),
            'drop': str(preds['drop']).replace(' ', '')
        })
    
    # Сохранение в CSV
    pd.DataFrame(submission).to_csv(output_path, index=False)
    print(f"Сабмит сохранен в {output_path}")


In [ ]:
# Путь к тестовым данным
TEST_PATH = os.path.join(DATA_PATH, "test")

# Получаем список всех файлов в тестовой папке
test_files = [os.path.join(TEST_PATH, f) for f in os.listdir(TEST_PATH) 
              if os.path.isfile(os.path.join(TEST_PATH, f))]

# Генерация сабмита
generate_submission(
    model=model,
    test_files=test_files,
    output_path="../data/submissions/submission_model+_max1drop_max1recovery_Window60.csv"
)

In [ ]:
def remove_drops(submission_path, output_path):
    """
    Удаляет все интервалы drop из сабмита и сохраняет результат в новый файл.
    
    :param submission_path: Путь к исходному файлу сабмита.
    :param output_path: Путь для сохранения нового файла без drop.
    """
    # Загрузка сабмита
    submission = pd.read_csv(submission_path)
    
    # Удаление всех интервалов drop
    submission['drop'] = submission['drop'].apply(lambda x: '[]')
    
    # Сохранение результата
    submission.to_csv(output_path, index=False)
    print(f"Сабмит без drop сохранен в {output_path}")

In [ ]:
remove_drops("../data/submissions/submission_model+_max2drop.csv", "../data/submissions/submission_model+_without_drop.csv")